In [ ]:
! pip install request
! pip install json 
! pip install os

In [21]:
import requests
import json
import os

# Replace with your ACN-Data API token
TOKEN = "Dqj06kuBFgUFcD0gxSQ3Mr1FtP5lPNFJJJJr9DVyhFg"

# Example site ID
site_id = "caltech"

# Directory and filename where we'll save the filtered JSON
save_directory = "/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR"
os.makedirs(save_directory, exist_ok=True)
save_path = os.path.join(save_directory, "sessions_filtered_2017.json")

# Set up an authenticated session (token as username, empty password)
session = requests.Session()
session.auth = (TOKEN, "")

# Filtering on date range:
# We will include &page=X for pagination
base_filter_url = (
    f"https://ev.caltech.edu/api/v1/sessions/{site_id}?"
    'where=connectionTime>="Sun, 1 Jan 2017 00:00:00 GMT" '
    'and connectionTime<="Sun, 31 Dec 2017 00:00:00 GMT"'
    "&pretty"
)

# The fields you want to keep from each session
wanted_keys = [
    "_id", "clusterID", "connectionTime", "disconnectTime", "doneChargingTime",
    "kWhDelivered", "sessionID", "siteID", "spaceID", "stationID", 
    "timezone", "userID", "userInputs"
]

# We'll accumulate all items from all pages here
all_items = []

page = 1
while True:
    # Build the URL with the desired page
    url = f"{base_filter_url}&page={page}"

    # Make the request
    response = session.get(url)
    if response.status_code != 200:
        print(f"Request failed on page {page} with status code {response.status_code}: {response.text}")
        break

    data = response.json()

    # Filter each item to keep only wanted keys
    items_this_page = []
    for item in data.get("_items", []):
        filtered_item = {k: item.get(k) for k in wanted_keys}
        items_this_page.append(filtered_item)

    # Add this page's items to the master list
    all_items.extend(items_this_page)

    # Check if there's a "next" link
    links = data.get("_links", {})
    next_link = links.get("next")
    if not next_link:
        # No more pages
        break

    # Move to the next page
    page += 1

# Prepare the final JSON structure
final_data = {
    "_items": all_items,
    # Optionally keep just top-level metadata from the last request
    "_links": data.get("_links", {}),
    "_meta": data.get("_meta", {})
}

# Save the filtered data to JSON
with open(save_path, "w") as f:
    json.dump(final_data, f, indent=4)

print(f"Retrieved a total of {len(all_items)} items across all pages.")
print(f"Filtered 2017 sessions saved to: {save_path}")


Retrieved a total of 0 items across all pages.
Filtered 2017 sessions saved to: /Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2017.json


In [16]:
import json
import pandas as pd
import os

# Paths (adjust as needed)
load_directory = "/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR"
json_filename = "sessions_filtered_2021.json"
csv_filename = "sessions_filtered_2021.csv"

json_path = os.path.join(load_directory, json_filename)
csv_path = os.path.join(load_directory, csv_filename)

# Load the existing JSON file
with open(json_path, "r") as f:
    data = json.load(f)

# Extract the list of sessions from the "_items" key
sessions = data.get("_items", [])

# Convert the sessions list to a DataFrame
df = pd.DataFrame(sessions)

# If "userInputs" is a list of dicts, convert each entry to a string for CSV storage
if "userInputs" in df.columns:
    df["userInputs"] = df["userInputs"].apply(lambda x: json.dumps(x) if x else None)

# Write the DataFrame to CSV
df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}")

CSV file saved to: /Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/sessions_filtered_2021.csv
